### Dopamine decay time constant estimation

Cragg SJ, Hille CJ, Greenfield SA. Dopamine release and uptake dynamics within nonhuman primate striatum in vitro. J Neurosci. 2000;20(21):8209-8217. doi:10.1523/JNEUROSCI.20-21-08209.2000

Data from marmoset.

Link: https://www.jneurosci.org/content/20/21/8209



![alt text](cragg2000-fig3A.jpg "Cragg2000-fig3A")

In [1]:
import numpy as np

import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'iframe'
pio.templates.default = 'simple_white'

In [2]:
def cubic_interpolate(time, values, dt=1e-3):
    from scipy.interpolate import CubicSpline 

    t, v = time, values # shorthand
    cs = CubicSpline(x=t, y=v)
    
    t = np.arange(start=np.min(t), stop=np.max(t), step=dt)
    
    return t, cs

def mask_crossing(arr, threshold):
    mask = np.array(arr > threshold, dtype=np.int8)
    mask[1:] = np.diff(mask)
    mask[0] = 0

    return mask

In [3]:
# digitized data, using fig 3A from Cragg et al. 2000
original_time=np.array([1.2660550458715598, 1.4999999999999998, 1.7339449541284402, 2.0091743119266052, 2.229357798165138, 2.5045871559633026, 2.738532110091744, 3, 3.2477064220183482, 3.4954128440366983, 0.23394495412844019, 0.5091743119266054, 0.7293577981651376, 0.9908256880733946])
original_concentration=np.array([1.8634538152610438, 1.6626506024096384, 1.0200803212851401, 0.618473895582329, 0.4497991967871484, 0.3373493975903612, 0.25702811244979884, 0.20080321285140545, 0.17670682730923692, 0.1686746987951806, -0.008032128514056325, -0.008032128514056325, 0.008032128514056325, 0.2730923694779117])

# unsorted when digitzed
idxs = np.argsort(original_time)
original_time=original_time[idxs]
original_concentration=original_concentration[idxs]

# use spline interpolation
time, cs = cubic_interpolate(time=original_time, values=original_concentration, dt=1e-3)
concentration = cs(time,0)

# get y-axis crossing of 1/exp(t/tau) where t=tau
mask=mask_crossing(concentration, np.exp(-1))
cross_idx = np.min(np.where(mask==-1)[0])
peak_idx = np.argmax(concentration)
tau=time[cross_idx] - time[peak_idx]




In [4]:

# plot
fig=go.Figure()
fig.add_trace(go.Scatter(x=original_time, y=original_concentration, mode='markers', name="raw data (Cragg 2000 fig3A)", marker=dict(size=15, color='red')))
fig.add_trace(go.Scatter(x=time, y=concentration, mode='lines', name="interpolated", line=dict(color='black')))
fig.add_trace(go.Scatter(x=time[[peak_idx, cross_idx]], y=concentration[[peak_idx, cross_idx]], marker=dict(size=15, color='black'), mode='markers', name='idxs'))
fig.update_layout(title=f"tau = {tau:.3f} (s)", xaxis_title='time (s)', yaxis_title='concentration (uM)')

